# Milestone 2: Preprocessing

**Import Libraries**

In [2]:
from collections import Counter
import math
import time

import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from scipy import stats
from sklearn.linear_model import Lasso, LassoCV
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_validate
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures
from sklearn.utils import shuffle

from warnings import simplefilter
simplefilter('ignore', category=FutureWarning)

## 1. Access the data that you will be using for the final project by downloading, collecting, or scraping* from the relevant source(s)

We accessed **fertility rate by state over time** (the response variable) from https://www.cdc.gov/nchs/pressroom/sosmap/fertility_rate/fertility_rates.htm and saved it as `fertility_rate_census.csv` DEFINED AS: 

We care about basic demographic data which we will get from the census. We will only include data for women from ages 15-44 and then aggregate by state. We will then create a new dataset of percentage of each demographic for each state. Ex: percentage of white (women 15-44) in MA, percentage of asian in MA,... percentage of white in NJ, percentage of asian in NJ etc. 

Here is the census data we consider important (based on our own subjective opinions):
- race
- socio economic status (household income) - percentage of households below the poverty line
- education level - share that are high school graduates 
- immigration status

And obviosuly we need **age**, **sex** , **year** , and **state**  in order to aggregate our data

We got this data from IPSUMS USA, downloading it and importing it into the notebook

We also care about religion, political makeup, and whether certain abortion laws are in place (all of which are not in the census). We will find them different ways. Please note **WE ARE TRYING TO USE AS FEW CATEGORICAL DATAPOINTS AS POSSIBLE** as we don't want our predictions to get too specific. 

## 2. Load the data into a Jupyter notebook and understand the data by examining, among other characteristics of interest, data missingness, imbalance, and scaling issues.

Some issues we preliminary have considered before even inspecting the data includes:


- **under reporting immigration status**: via google, people tend to underreport whether they are immigrants. This is potentially a missingness issue
-  **Multicollinearity**: There is most likely a relationship between racial background / household income and an individual's birth country (immigration status) so we can't use both as predictors in the same equation. We don't know how strong this correlation will be so are not worried yet, but would love to talk about it with a TF


- **How to determine political makeup**: 

- **is this just too much data??**: looking at each of these attributes for each state for each year may just be too many dimensions. Is there really a big difference accross years? Should we just look at 1? If so, which? 

- **Abortion Laws**: Some of the literature states that abortion laws have an unintuitive effect on fertility rates. We've decided to create a variable that captures the "severity" of the abortion laws for each state (ex. 0 for nothing, 0.3 for 3 weeks, 0.65 for 5 weeks etc.).

Now we are going to import and inspect each dataset, looking for missingness, imbalance, and scaling issues!

In [3]:
fertility_rate = pd.read_csv('data/fertility_rate_census.csv')
fertility_rate.head()

,YEAR,STATE,FERTILITY RATE,BIRTHS,URL
0,2022,AL,58.7,58149,/nchs/pressroom/states/alabama/al.htm
1,2022,AK,64.9,9359,/nchs/pressroom/states/alaska/ak.htm
2,2022,AZ,54.9,78547,/nchs/pressroom/states/arizona/az.htm
3,2022,AR,60.2,35471,/nchs/pressroom/states/arkansas/ar.htm
4,2022,CA,52.8,419104,/nchs/pressroom/states/california/ca.htm


## 3. Understand and describe the preprocessing required such that data is in a form amenable to later downstream tasks such as visualizing and modeling, as is appropriate to the specific project goals.

